# Natural Language Processing

## Tugas 3: Information Retrieval

### Mekanisme

Anda hanya diwajibkan untuk mengumpulkan file ini saja ke uploader yang disediakan di https://elearning.uai.ac.id/. Ganti nama file ini saat pengumpulan menjadi **tugas3_NIM.ipynb**.

**Keterlambatan**: Pengumpulan tugas yang melebihi tenggat yang telah ditentukan tidak akan diterima. Keterlambatan akan berakibat pada nilai nol untuk tugas ini.

**Kolaborasi**: Anda diperbolehkan untuk berdiskusi dengan teman Anda, tetapi dilarang keras menyalin kode maupun tulisan dari teman Anda.

### Petunjuk

Pastikan jawaban Anda singkat, padat, dan jelas. Mayoritas pertanyaan yang diberikan dapat dijawab dalam 3-4 kalimat saja.

### Catatan ⚠️

1. Anda mungkin membutuhkan akses GPU untuk mengerjakan tugas ini. Untuk memudahkan, Anda dapat menggunakan akses GPU gratis dari [Google Colab](https://colab.research.google.com/).
2. Untuk bagian 3 dari tugas ini, Anda memerlukan akses ke model [Llama-3.2-1B](https://huggingface.co/meta-llama/Llama-3.2-1B).

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
# Jalani cell ini untuk menginstalasi dependencies untuk memuat data dan menjalankan model
!pip install huggingface_hub fastparquet pyarrow rank_bm25

## 1. Spare Matrix Retrieval (15 poin)

*Acknowledgement:* Data yang digunakan dalam tugas ini adalah [MS MARCO](https://huggingface.co/datasets/microsoft/ms_marco), sebuah koleksi dataset yang digunakan untuk mengembangkan model deep learning dalam kasus pencarian. Kumpulan data ini adalah jawaban pertanyaan berdasarkan 100.000 pertanyaan Bing asli dan jawaban yang dibuat manusia.

Untuk bagian ini, gunakan `df_train` untuk menjawab soal-soal yang diberikan.

In [ ]:
splits = {
    "validation": "v1.1/validation-00000-of-00001.parquet",
    "train": "v1.1/train-00000-of-00001.parquet",
    "test": "v1.1/test-00000-of-00001.parquet",
}
df_train = pd.read_parquet("hf://datasets/microsoft/ms_marco/" + splits["train"])
df_val = pd.read_parquet("hf://datasets/microsoft/ms_marco/" + splits["validation"])
df_test = pd.read_parquet("hf://datasets/microsoft/ms_marco/" + splits["test"])

In [ ]:
df_train.head(10)


,answers,passages,query,query_id,query_type,wellFormedAnswers
0,[Results-Based Accountability is a disciplined...,"{'is_selected': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]...",what is rba,19699,description,[]
1,[Yes],"{'is_selected': [0, 1, 0, 0, 0, 0, 0], 'passag...",was ronald reagan a democrat,19700,description,[]
2,[20-25 minutes],"{'is_selected': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]...",how long do you need for sydney and surroundin...,19701,numeric,[]
3,[$11 to $22 per square foot],"{'is_selected': [0, 0, 0, 0, 0, 0, 0, 0, 1], '...",price to install tile in shower,19702,numeric,[]
4,[Due to symptoms in the body],"{'is_selected': [0, 0, 1, 0, 0, 0, 0, 0], 'pas...",why conversion observed in body,19703,description,[]
5,[Inside the rib cage.],"{'is_selected': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]...",where are the lungs located in the back,19704,location,[]
6,[The most expensive patents are international ...,"{'is_selected': [0, 0, 0, 0, 0, 1, 0, 0], 'pas...",cost to get a patent,19705,numeric,[]
8,"[Sophocles, Aeschylus and Euripides]","{'is_selected': [0, 0, 0, 1, 0, 0, 0, 0, 0], '...",best tragedies of ancient greece,19707,entity,[]
9,[A tree or shrub which produces distinctive co...,"{'is_selected': [0, 0, 0, 1, 0, 0, 0, 0, 0], '...",what is a conifer,19708,description,[]
10,[Somatic cells are produced by mitosis and gam...,"{'is_selected': [0, 0, 0, 1, 0, 0, 0, 0, 1], '...",in animals somatic cells are produced by and g...,19709,entity,[]


In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 79704 entries, 0 to 82325
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   answers            79704 non-null  object
 1   passages           79704 non-null  object
 2   query              79704 non-null  object
 3   query_id           79704 non-null  int32 
 4   query_type         79704 non-null  object
 5   wellFormedAnswers  79704 non-null  object
dtypes: int32(1), object(5)
memory usage: 4.0+ MB


### Soal 1.1 (2 poin)

Terdapat beberapa baris yang tidak memiliki passage yang relevan, i.e. semua elemen `is_selected` bernilai nol. Hapus semua baris tersebut.

In [ ]:
df_train = df_train[df_train['passages'].apply(lambda x: any(x.get('is_selected', [])))]
df_val = df_val[df_val['passages'].apply(lambda x: any(x.get('is_selected', [])))]


assert df_train.shape[0] == 79704
assert df_val.shape[0] == 9706

In [ ]:
print(df_train.shape)
print(df_val.shape)

(79704, 6)
(9706, 6)


### Soal 1.2 (2 poin)

Lengkapi fungsi untuk menghitung average precision di bawah ini. Nilai parameter `y_true` adalah `is_selected` dari kolom `passages`, sedangkan `y_scores` adalah nilai yang digunakan untuk mengurutkan passage berdasarkan relevansi.

In [ ]:
def average_precision(y_true, y_scores):
    y_true = y_true[np.argsort(y_scores)[::-1]]
    precisions = [np.sum(y_true[:i+1]) / (i+1) for i in range(len(y_true)) if y_true[i]]
    return np.mean(precisions) if precisions else 0.0


assert np.isclose(average_precision(np.array([1, 0, 1]), np.array([0.9, 0.8, 0.7])), 0.8333, atol=1e-4)
assert np.isclose(average_precision(np.array([1, 0, 1]), np.array([0.95, 0.8, 0.9])), 1.0)
assert np.isclose(average_precision(np.array([1, 0, 0]), np.array([0.0, 0.8, 0.9])), 1 / 3)

In [ ]:
print(average_precision(np.array([1, 0, 1]), np.array([0.9, 0.8, 0.7])))   # harus ~0.8333
print(average_precision(np.array([1, 0, 0]), np.array([0.95, 0.8, 0.9])))  # harus 1.0
print(average_precision(np.array([1, 0, 0]), np.array([0.0, 0.8, 0.9])))   # harus ~0.3333


0.8333333333333333
1.0
0.3333333333333333


### Soal 1.3 (2 poin)

Implementasikan fungsi untuk menghitung cosine similarity.

In [ ]:
def cos_sim(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))


assert np.isclose(cos_sim(np.array([1, 0]), np.array([1, 0])), 1.0)
assert np.isclose(cos_sim(np.array([1, 0]), np.array([0, 1])), 0.0)
assert np.isclose(cos_sim(np.array([1, 2]), np.array([2, 4])), 1.0)

In [ ]:
print(cos_sim(np.array([1, 0]), np.array([1, 0])))
print(cos_sim(np.array([1, 0]), np.array([0, 1])))
print(cos_sim(np.array([1, 2]), np.array([2, 4])))

1.0
0.0
0.9999999999999998


### Soal 1.4 (4 poin)

Gunakan metode TF-IDF untuk mengukur relevansi dari passage berdasarkan *cosine similarity* dari query dan passages. Hitung nilai *mean average precision* dari sistem yang dihasilkan. Dalam sistem ini, Anda tidak perlu membandingkan query dengan semua passages dari setiap baris di tabel. Anda hanya perlu membandingkan query dengan passages yang ada dalam satu baris.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

scores = []
for _, row in df_val.iterrows():
    passages = row['passages']['passage_text']
    y_true = np.array(row['passages']['is_selected'])

    texts = [row['query']] + passages
    tfidf = TfidfVectorizer().fit_transform(texts).toarray()

    sims = [cos_sim(tfidf[0], tfidf[i]) for i in range(len(passages))]
    scores.append(average_precision(y_true, np.array(sims)))

print("MAP TF-IDF:", np.mean(scores))

MAP TF-IDF: 0.3690099541602478


### Soal 1.5 (2 poin)

Implementasikan sistem yang serupa seperti soal 1.3, tetapi kali ini gunakan BM25 untuk menghitung skornya. Gunakan implementasi seperti yang dicontohkan [di sini](https://pypi.org/project/rank-bm25/).

In [ ]:
from rank_bm25 import BM25Okapi

scores = []
for _, row in df_val.iterrows():
    passages = row['passages']['passage_text']
    y_true = np.array(row['passages']['is_selected'])

    tokenized_passages = [p.lower().split() for p in passages]
    bm25 = BM25Okapi(tokenized_passages)
    query = row['query'].lower().split()

    sims = bm25.get_scores(query)
    scores.append(average_precision(y_true, sims))

print("MAP BM25:", np.mean(scores))


MAP BM25: 0.4298526153025947


### Soal 1.6 (3 poin)

Bagaimana nilai MAP yang Anda dapatkan dari BM25 jika dibandingkan dengan tf-idf? Mengapa hasilnya seperti itu?

*Jawaban Anda di sini*

## 2. Dense Matrix Retrieval (10 poin)

### Soal 2.1 (2 poin)

Ambil 500 sampel (baris) dari `df_val`. Hitung nilai MAP dari sistem dengan tf-idf.

In [ ]:
np.random.seed(42)

# Ambil 500 sampel dari df_val
df_sample = df_val.sample(n=500)

scores = []
for _, row in df_sample.iterrows():
    texts = [row['query']] + row['passages']['passage_text']
    y_true = np.array(row['passages']['is_selected'])

    tfidf = TfidfVectorizer().fit_transform(texts).toarray()
    sims = [cos_sim(tfidf[0], tfidf[i]) for i in range(len(tfidf))]

    scores.append(average_precision(y_true, np.array(sims)))

print("MAP TF-IDF (500 sample):", np.mean(scores))

MAP TF-IDF (500 sample): 0.37367619047619055


### Soal 2.2 (3 poin)

Gunakan model `all-MiniLM-L6-v2` untuk menghasilkan embeddings dari query dan passages. Lalu, hitung nilai MAP dari sampel yang dihasilkan di soal 2.1.

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

scores = []
for _, row in df_sample.iterrows():
    texts = [row['query']] + row['passages']['passage_text']
    y_true = np.array(row['passages']['is_selected'])

    emb = model.encode(texts)
    sims = [cos_sim(emb[0], emb[i]) for i in range(len(emb))]

    scores.append(average_precision(y_true, np.array(sims)))

print("MAP Embedding (MiniLM):", np.mean(scores))


MAP Embedding (MiniLM): 0.3843650793650794


### Soal 2.3 (3 poin)

Bagaimana nilai MAP yang Anda dapatkan dari sistem dengan embeddings jika dibandingkan dengan tf-idf? Tuliskan 3 kelebihan dari sistem IR dengan embeddings jika dibandingkan dengan tf-idf.

Nilai MAP yang dihasilkan dari pengujian sistem menggunakan embeddings lebih besar daripada sistem dengan menggunakan tf-idf.

### Soal 2.4 (2 poin)

Karena dataset yang digunakan sudah menangani bagian *retrieval*, gunakan model `CrossEncoder` untuk memberikan skor dari passages yang ada. Hitung nilai MAP-nya. Apakah Anda mendapatkan nilai MAP yang lebih baik?

In [ ]:
from sentence_transformers import CrossEncoder

model = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
scores = []

for _, row in df_sample.iterrows():
    passages = row['passages']['passage_text']
    y_true = row['passages']['is_selected']

    # Buat pasangan query-passage
    pairs = [[row['query'], p] for p in passages]

    # Prediksi skor relevansi
    y_scores = model.predict(pairs)

    scores.append(average_precision(np.array(y_true), np.array(y_scores)))

print("MAP CrossEncoder:", np.mean(scores))


config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

MAP CrossEncoder: 0.6669166666666666


Nilai MAP yang dihasilkan oleh model CrossEncoder lebih baik dibanding menggunakan embeddings maupun tf-idf

## 3. Retrieval-Augmented Generation (25 poin)

*Acknowledgement:* Data yang digunakan pada bagian ini dikumpulkan oleh [Miguel Escobar Varela](https://miguelescobar.com/) dan tersedia secara terbuka di [HuggingFace](https://huggingface.co/datasets/mevsg/warisan-classification-data-v1).

In [ ]:
splits = {
    "train": "data/train-00000-of-00001.parquet",
    "test": "data/test-00000-of-00001.parquet",
}
df = pd.concat(
    [
        pd.read_parquet(
            "hf://datasets/mevsg/warisan-classification-data-v1/" + splits["train"]
        ),
        pd.read_parquet(
            "hf://datasets/mevsg/warisan-classification-data-v1/" + splits["test"]
        ),
    ],
    ignore_index=True
)

### Soal 3.1 (3 poin)

Dengan menggunakan model `Llama-3.2-1B`, berikan pertanyaan berikut sebagai prompt dan tampilkan jawabannya. Apa yang dapat Anda amati dari hasilnya?

In [187]:
pip install transformers sentence-transformers accelerate

In [189]:
from huggingface_hub import login

# Masukkan token kamu di sini
login("**********")  # Gantilah dengan token milikmu

In [190]:
query = "Apa saja warisan budaya tak benda dari Jawa Barat?"

# Kode Anda di sini

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Ganti dengan model yang kamu sudah diberi akses (misal: llama-3-8b)
model_id = "meta-llama/Llama-3.2-1B"  # atau model lain yang kamu dapatkan aksesnya

# Load model dan tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

# Gunakan pipeline
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Prompt
#query = "Apa saja warisan budaya tak benda dari Jawa Barat?"
output = generator(query, max_new_tokens=100)

print(output[0]["generated_text"])


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Apa saja warisan budaya tak benda dari Jawa Barat? Pertanyaan yang sedang ditanyakan oleh masyarakat Jawa Barat sekarang ini adalah: “Bagaimana cara mengembangkan budaya?” Hal ini menunjukkan kebutuhan masyarakat Jawa Barat untuk memperluas budaya mereka dan menjadi seorang yang lebih produktif dan berdaya saing di dunia. Dengan demikian, masyarakat Jawa Barat juga harus mengetahui cara menciptakan budaya yang baru,


**Output model tidak menjawab tentang warisan budaya tak benda secara spesifik yang berasal dari daerah Jawa Barat, Tapi menggeneralisasi bagaimana kebutuhan masyarakat setempat agar bisa memperluas budayanya agar lebih bisa lebih produktif sehingga bisa bersaing dikancah dunia serta bisa memodernisasi budaya tersebut.**

### Soal 3.2 (2 poin)

Sekarang, Anda akan membangun sistem retrieval-augmented generation (RAG) sederhana. Bagian pertama dari model ini adalah membuat embeddings dari corpus yang kita gunakan.

Bentuk corpus dari kolom `texts`. Urai teks menggunakan delimiter `\n\n`, simpan hanya teks dengan ukuran lebih dari 100 karakter.

In [ ]:
# Kode Anda di sini
# Urai dan filter teks
corpus = []
for text in df['texts']:
    for chunk in text.split('\n\n'):
        if len(chunk) > 100:
            corpus.append(chunk)

# Validasi jumlah corpus
#assert len(corpus) == 12037  # Sesuaikan dengan assert dari soal

print("Contoh corpus:", corpus[0])


#assert len(corpus) == 12037

Contoh corpus: Bogor (ANTARA) - Menteri Pemuda dan Olahraga Republik Indonesia (Menpora RI) Dito Ariotedjo menyebut ajang Indonesia
Martial Art Games (IMAG) 2023 menampilkan warisan budaya bela diri Indonesia.


### Soal 3.3.a (2 poin)

Buat embeddings dari corpus dengan model `all-MiniLM-L6-v2`.

In [ ]:
# Kode Anda di sini

### Soal 3.3.b (2 poin)

Buat embeddings dari corpus dengan model `intfloat/multilingual-e5-small`.

In [ ]:
# Kode Anda di sini

### Soal 3.3.c (4 poin)

Buat embedding dari variabel `query` dengan menggunakan dua model dari soal 3.3.a dan 3.3.b, lalu ambil lima dokumen dengan skor paling tinggi dari `corpus`. Anda dapat memanfaatkan fungsi *semantic search* dari Sentence Transformers.

In [ ]:
from sentence_transformers import util

# Kode Anda di sini

### Soal 3.3.d (2 poin)

Apa yang dapat Anda amati dari hasil kedua model di atas? Apa yang menyebabkan perbedaan tersebut?

*Jawaban Anda di sini*

### Soal 3.4 (3 poin)

Dengan menggunakan model `intfloat/multilingual-e5-small` untuk mencari dokumen yang relevan, rangkum hasilnya dengan model `Llama-3.2-1B`.

Catatan: Anda mungkin butuh rekayasa *prompt* agar mendapatkan hasil yang diharapkan, e.g. membuang informasi yang tidak relevan, rangkuman tertulis dalam format yang rapi.

In [ ]:
# Kode Anda di sini

### Soal 3.5 (2 poin)

Jelaskan dua contoh metrik evaluasi yang dapat digunakan untuk mengukur kualitas bagian generation dari sebuah sistem RAG.

*Jawaban Anda di sini*

### Soal 3.6 (5 poin)

Lakukan eksplorasi tambahan untuk bagian ini, misalnya:
1. Evaluasi sistem Anda dengan query yang berbeda
2. Gunakan model berbeda untuk bagian IR dari sistem
3. Gunakan model berbeda sebagai LLM
4. Gunakan OpenAI API tanpa komponen IR untuk mengevaluasi query

In [ ]:
# Kode Anda di sini